In [2]:
from diffractio import degrees, mm, plt, sp, um, np
from diffractio.scalar_masks_XY import Scalar_mask_XY
from diffractio.scalar_masks_XYZ import Scalar_mask_XYZ
from diffractio.scalar_sources_XY import Scalar_source_XY

from scipy.constants import c, epsilon_0, e, m_e, h
import numpy as np
import sympy as sp

from sympy.core.numbers import pi
from IPython.display import display
import numba
from numbalsoda import dop853, lsoda, lsoda_sig
from numba import njit, cfunc

%matplotlib widget


Here we ware going to use a light optics package to calculate the path of the electron. We can do this because in purely electrostatic systems, there is an electrostatic analogue of snells law. We can use this to test if the optical path length calculations of our ray tracing model actually work. We will test it again at a scale that for now is not physically accurate. If our ray tracing model recreates the fringe spacing, then we know that our optical path length calculations will work. 

In [11]:
X, Y, Z = sp.symbols('X Y Z')

#Use scipy's constants
e = e #-1.60217662e-19 #unit C
m = m_e #9.10938356e-31 #unit kg
h_planck = h #6.62607015e-34 #unit J . s 
c_l = c #2.99792458e8 #unit m/s
eps_0_val = epsilon_0 #8.85418782e-12 #permittivity unit F/m
#eps = abs(e)/(2*m*c_l**2) #eps as defined in Hawkes, different from the permittivity of free space. 

#Set initial accelerating voltage. I know it's unrealistic but it gives an output that is easy to plot for now.
phi_0 = 0.5

#Calculate corresponding wavelength of the electron, and divide by 1e-6 so that everything is scaled around the micron unit (this is for the diffractio package)
wavelength = h/(2*abs(e)*m*phi_0)**(1/2)/1e-6

#Set voltage of biprism
V = 0.1

#Set radius of biprism wire
r = 0.125e-6/1e-6

#Set radius of the condenser surrounding the wire
R = 1e-5/1e-6

phi_biprism = V*(sp.log((X*X+(Z)**2)/(R**2)))/(2*sp.log(r/R))

#Here we write the "refractive index" that the electron would see, using the equation for the electrostatic potential. See Eq 5.17 in Hawkes Principles of Electron Optics
n_eq1 = (1+phi_biprism/phi_0)**(0.5)
n_eq1_lambda = sp.lambdify([Z, X, Y], n_eq1)

#The equation needs to be written as a string for diffractio to be able to implement it, and we need to replace log with np.log in the expression.
#This is because diffractio uses the "eval" function which uses a string to form the function.
n = str(n_eq1).replace("log", "np.log")

#Set up the scale in x and z
x0 = np.linspace(-0.4 * um, 0.4 * um, 512 * 1)
y0 = np.linspace(-0.4 * um, 0.4 * um, 512 * 1)
z0 = np.linspace(-2 * um, 3 * um, 512 * 1)
zz0, xx0, yy0 = np.meshgrid(z0, x0, y0)

#Create a source plane wave
u0 = Scalar_source_XY(x=x0, y=y0, wavelength=wavelength)
u0.plane_wave(A=1, z0=0 * um, theta=0. * degrees)

t0 = Scalar_mask_XYZ(x=x0, y=y0, z=z0, wavelength=wavelength, n_background=0)
t0.incident_field(u0)

'''
#Create a condenser sphere that has the refractive index of a biprism
t0.sphere(r0=(0, 0, 0),
             radius= R * um,
             refraction_index=n,
             angles = (0, 0, 0))

#Create a model of the wire, with a complex value that represents "absorbtion". This should enable 
#the wave to diffract around the wire
t0.sphere(r0=(0, 0, 0),
          radius=r * um,
          refraction_index=0 + 10j,
          angles = (0, 0, 0))
'''
#Create the biprism volume
t0.n = np.complex128(n_eq1_lambda(zz0, xx0, yy0))

t0.sphere(r0=(0, 0, 0),
          radius=r * um,
          refraction_index=0 + 10j,
          angles = (0, 0, 0))



TypeError: float() argument must be a string or a real number, not 'complex'

Plot the results

In [ ]:
#Perform the calculation with the wave propagation method
t0.WPM(verbose=True, has_edges=False)

In [ ]:
t0.draw(kind='intensity', draw_borders=True)

In [ ]:
t0.draw(kind='phase', draw_borders=True)

In [ ]:
u_field_last_plane = t0.profile_transversal(z0= 3 * um, normalize = 'maximum')
wave_ax = plt.gca()
wave_fig = plt.gcf()
plt.xlim([-0.08, 0.08])

Now we have a baseline, we should be able to recreate it with our optical path length ray tracing code, and will achieve the same fringe spacing if our code works.

Again setup all equations and functions.

In [ ]:
def make_potential_and_efield(phi, phi_0, x, y, z):
    
    phi_hat = (phi_0+phi)#*(1+eps*(phi_0-phi))
    
    #Get E field function. Notice there is no negative sign, as the way hawkes uses phi_hat, there is no minus before it (Equation 3.22).
    dphi_hat_wires_electron_dx = phi_hat.diff(x)
    dphi_hat_wires_electron_dy = phi_hat.diff(y)
    dphi_hat_wires_electron_dz = phi_hat.diff(z)
    phi_hat_wires_electron_lambda = numba.jit(sp.lambdify([x, y, z], phi_hat, 'numpy'), nopython = True)
    dphi_hat_wires_electron_lambda = numba.jit(sp.lambdify([x, y, z], [dphi_hat_wires_electron_dx, dphi_hat_wires_electron_dy, dphi_hat_wires_electron_dz], 'numpy'), nopython = True)
    
    return phi_hat_wires_electron_lambda, dphi_hat_wires_electron_lambda

phi_hat_lambda, dphi_hat_lambda = make_potential_and_efield(phi_biprism, phi_0, X, Y, Z)

@cfunc(lsoda_sig)
def trajectory_equation_of_motion_numba(z, x, dx, phi_hat_0):
    
    p = (1+x[1]**2+x[3]**2)**(1/2) #obtain slope of ray
    phi_hat = phi_hat_lambda(x[0], x[2], z) #evaluate potential
    dphi_hat_x, dphi_hat_y, dphi_hat_z = dphi_hat_lambda(x[0], x[2], z) #evaluate the efield
    dx[0] = x[1]
    dx[1] = (p**2)/(2*phi_hat)*(dphi_hat_x-x[1]*dphi_hat_z)
    dx[2] = x[3]
    dx[3] = (p**2)/(2*phi_hat)*(dphi_hat_y-x[3]*dphi_hat_z)
    dx[4] = (phi_hat/phi_hat_0[0])**(1/2)*p

In [ ]:
macro_block_size = 64
n_threads = 12
scale = 1
n = (n_threads*macro_block_size*scale)**2
num_along_dim = int(np.sqrt(n))
print(n)
print(num_along_dim)

In [ ]:
x0 = np.concatenate((np.linspace(-0.4, -0.126, num_along_dim//2, dtype = np.float64, endpoint=True), np.linspace(0.126, 0.4, num_along_dim//2, dtype = np.float64, endpoint=True)))
y0 = np.linspace(0, 0, num_along_dim, dtype = np.float64, endpoint=True)

grid = np.array(np.meshgrid(x0, y0))
pos_0 = grid.reshape(2, n).T

z_eval = np.array([-2., 0, 3])

#Need the potential at the beginning for the optical path length calculation.
phi_hat_lambda_0 = phi_hat_lambda(0, y0[0], z_eval[0])

#We need to assign the equation for numbalsoda to an address in memory.
rhs = trajectory_equation_of_motion_numba
funcptr = rhs.address

In [ ]:
from numba_progress import ProgressBar

@numba.njit(nogil=True, parallel=True)
def main(n, pos_0, phi_hat_lambda_0, progress_proxy):
    
    x_out = np.empty((n,len(z_eval)), np.float64)
    y_out = np.empty((n,len(z_eval)), np.float64)
    opl_out = np.empty((n,len(z_eval)), np.float64)
    n_threads = 12 #Number of threads on laptop cpu
    thread_block_size = int(n/n_threads)
    macro_block_size = 64
    macro_blocks = int(thread_block_size/macro_block_size)
    
    for thread in numba.prange(n_threads):
        thread_idx = thread*thread_block_size
        
        for macro_block in range(macro_blocks):
                #get starting index for the block of rays of each macroblock in a thread.
                macro_block_idx = macro_block*macro_block_size
                
                #combine the starting index of the thread and macroblock
                idx_start = thread_idx + macro_block_idx
                
                for j in range(macro_block_size):
                    u0 = np.empty((5,), np.float64)
                    
                    u0[0] = pos_0[idx_start+j, 0]
                    u0[1] = 0.
                    u0[2] = pos_0[idx_start+j, 1]
                    u0[3] = 0.
                    u0[4] = 0.
                    
                    usol, success = lsoda(funcptr, u0, z_eval, data = phi_hat_lambda_0, rtol = 1.0e-13, atol = 1.0e-20)
                    
                    x_out[idx_start+j] = usol[:,0]
                    y_out[idx_start+j] = usol[:,2]
                    opl_out[idx_start+j] = usol[:,4]

                    progress_proxy.update(1)
                    
    return x_out, y_out, opl_out

with ProgressBar(total=n) as progress:
    x_out, y_out, opl_out = main(n, pos_0, np.array([phi_hat_lambda_0]), progress)
    

In [ ]:
fig = plt.figure()
ax = plt.axes(projection='3d')

def get_scale(x_out, y_out):
    x_left = np.argmin(x_out)
    x_right = np.argmax(x_out)
    y_bottom = np.argmin(y_out)
    y_top = np.argmax(y_out)
    
    return x_out[x_left], x_out[x_right], y_out[y_bottom], y_out[y_top]

def plot_extent(ax, x_out, y_out, z_eval, c):
    
    x_left, x_right, y_bottom, y_top = get_scale(x_out, y_out)
    
    ax.plot(x_left, y_bottom, z_eval, c)
    ax.plot(x_left, y_top, z_eval, c)
    ax.plot(x_right, y_top, z_eval, c)
    ax.plot(x_right, y_bottom, z_eval, c)
    ax.plot(x_out[::1000], y_out[::1000], z_eval, c)

    
plot_extent(ax, x_out[:, 0], y_out[:, 0], z_eval[0], '.r')
plot_extent(ax, x_out[:, 1], y_out[:, 1], z_eval[1], '.g')
plot_extent(ax, x_out[:, -1], y_out[:, -1], z_eval[-1], '.b')



In [ ]:
z_out = np.tile(z_eval,(x_out.shape))[:, 0:x_out.shape[1]]
fig, ax = plt.subplots()
ax.plot(x_out[::1000, :].T, z_out[::1000, :].T)

In [ ]:
import scipy 
from collections import defaultdict

def plot_interference(x_out, y_out, z_eval, phase_out, z_plane_idx, scale):

    x_left = -0.08 #scale[0]
    x_right = 0.08 #scale[1]

    x_edge = np.abs(x_left) + np.abs(x_right)
    
    pixels_x_edge = 200
    
    image = np.zeros((pixels_x_edge, 1), np.float64)

    pixel_size_x = (x_edge)/pixels_x_edge
    
    #Create a boolean mask to filter out values outside the valid range of the image plane.
    valid_mask_x = (x_out[:, z_plane_idx] > x_left) & (x_out[:, z_plane_idx] < x_right)

    phase_out_valid = phase_out[valid_mask_x]

    #Filter the coordinates to keep only the valid ones
    x_out_valid = x_out[valid_mask_x, z_plane_idx]

    #Create the pixel bins
    bins_x = np.arange(x_left, x_right+pixel_size_x, pixel_size_x)

    x_bin_indices = np.digitize(x_out_valid, bins_x) - 1

    bin_values = defaultdict(list)

    for i, (x_idx) in enumerate(x_bin_indices):
        bin_values[(x_idx, 0)].append(phase_out_valid[i])  

    #Loop through the coordinates, find the phases of all the rays in each pixel, and calculate the interference. 
    for x, y in bin_values:
        phases = np.array(bin_values[x, y])
        image[x, y] = np.abs(np.sum(np.exp(1j*(phases)))/len(phases))**2
        my_dpi = 300
    
    x_intensity = bins_x[:-1] + pixel_size_x/2

    intensity_fig, intensity_ax = plt.subplots()
    intensity_ax.plot(x_intensity, image[:, 0]/np.max(image), label = 'Intensity via OPD')
    intensity_ax.set_xlim(-0.08, 0.08)
    intensity_ax.set_xlabel('x (m)')
    intensity_ax.set_ylabel('I')
    intensity_ax.legend()

    return intensity_fig, intensity_ax

fractional_wavelength = opl_out[:, -1]/wavelength
phase_out = np.mod(fractional_wavelength*2*np.pi, 2*np.pi)
    
scale = get_scale(x_out[:, -1], y_out[:, -1])
opd_fig, opd_ax = plot_interference(x_out, y_out, z_eval, phase_out, -1, scale)



        

In [ ]:
opd_ax.plot(t0.x, u_field_last_plane/np.max(u_field_last_plane), label = 'Intensity via Wave Propagation Method as light wave')
plt.legend()